<a href="https://colab.research.google.com/github/Priyo-prog/Machine-Learning/blob/main/Principal%20Component%20Analysis/principal_components_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Principal Component Analysis**

* Create some gene data
* Use **PCA()** function from sklearn to do PCA
* Determine how much variation each principal components accounts for
* Draw a fancy PCA graph using matplotlib
* Examin the loading scores to determine what variables have the largest effect on the graph.

## Import important packages

In [ ]:
import pandas as pd
import numpy as np
import random as rd
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt

## Generate sample dataset

* The first thing we do is generate an array of 100 gene names.
* Since this is just an example we name them as "gene1", "gene2" etc.

In [ ]:
genes = ['gene' + str(i) for i in range(1,101)]

* Now we create arrays of sample names.
* We have 5 "wild type" or "wt" samples
* 5 "knock out" or "ko" samples

In [ ]:
wt = ['wt' + str(i) for i in range(1,6)]
ko = ['ko' + str(i) for i in range(1,6)]

Now we create a pandas dataframe to store the made up data

In [ ]:
# The "stars" unpack the "wt" and "ko" arrays so that column names are single array
data = pd.DataFrame(columns=[*wt, *ko], index=genes)

Finally creating the random data.

* For each gene in the "index", we create 5 values for "wt" samples and 5 values for "ko" samples
* The made up datacomes from two possion distribution: one for wt samples and one for ko samples